In [35]:
import pandas as pd
import pymysql.cursors
import psycopg2
import settings

def catch_error(func):
    def catch(*args, **kwargs):
        try:
            result = func(*args, **kwargs)
            if func.__name__ == 'query_db': return result 
        except Exception as e:
            print(e)
    return catch
    
def mysql_connection(kwargs):
    return pymysql.connect(**kwargs,
                             cursorclass=pymysql.cursors.DictCursor)

def postgres_connection(kwargs):
    return psycopg2.connect(**kwargs)
class Connect:
    def __init__(self, ql_type, **kwargs):
        self.params = kwargs
        self.ql_type = ql_type
        self.open_connection(ql_type)
    
    @classmethod
    def connectMySQL(cls, **kwargs):
        return cls('mysql', **kwargs)
    
    @classmethod
    def connectPostgresL(cls, **kwargs):
        return cls('postgres', **kwargs)   
    
    def open_connection(self,ql_type):
        if ql_type == 'mysql':
            self.connection = mysql_connection(self.params)
        else:
            self.connection = postgres_connection(self.params)

        self.cur = self.connection.cursor()
        
    def close(self):
        self.connection.close()
        
    @catch_error
    def exec_query(self,query):
        self.open_connection(self.ql_type)
        self.cur.execute(query) 
        self.connection.commit()
        self.close()

    @catch_error
    def query_db(self,query):
        self.open_connection(self.ql_type)
        self.cur.execute(query) 
        result = self.cur.fetchall()
        if not result:
            result = []     
        self.close()
        df = pd.DataFrame(result)
        if not df.empty:
            df.columns = [desc[0] for desc in self.cur.description]
        return df

class commonQuery(Connect):
    
    def drop_table(self, table):
        if type(table) == str:
            self.exec_query('DROP TABLE {}'.format(table))
        elif type(table) == list:
            for t in table:
                self.exec_query('DROP TABLE {}'.format(t))
        
    def delete(self, table):
        if type(table) == str:
            self.exec_query('DELETE FROM {}'.format(table))
        elif type(table) == list:
            for t in table:
                self.exec_query('DELETE FROM {}'.format(t))
    
    def count(self, table):
        count = self.query_db('SELECT count(*) as count FROM %s'%table)
        return count['count'][0]
    
    def head(self, table):
        head = self.query_db('SELECT * FROM %s LIMIT 5'%table)
        return head
    
    def show(self, table):
        show = self.query_db('SELECT * FROM %s'%table)
        return show
    
    def filter_equality(self, table, **equality):
        where = 'AND'.join(" {!s}={!r} ".format(key,val) for (key,val) in equality.items())
        result = self.query_db('SELECT * FROM %s WHERE %s'%(table, where))
        return result
    
    def show_tables(self):
        if self.ql_type == 'postgres':
            df = self.query_db("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")
        else:
            df = self.query_db('SHOW tables;')
        return df
    
    def describe(self, table):
        if self.ql_type == 'postgres':
            df = self.query_db("SELECT column_name, data_type, character_maximum_length from INFORMATION_SCHEMA.COLUMNS where table_name = '{}'".format(table))
        else:
            df = self.query_db("DESCRIBE %s"%table)
        return df
    
    def drop_table(self, table):
        self.exec_query('DROP TABLE {}'.format(table))

### Connect to MySQL

In [36]:
sql = commonQuery.connectMySQL(
                          host    = settings.host,
                          port    = settings.mysql_port,
                          user    = settings.mysql_user,
                          password= settings.mysql_password,
                          db      = settings.mysql_db)

### Connect to Postgres

In [37]:
pg = commonQuery.connectPostgresL(host    = settings.host,
                                 port    = settings.postgres_port,
                                 user    = settings.postgres_user,
                                 password= settings.postgres_password,
                                 dbname  = settings.postgres_db)